# Sistema de classificação de risco para concessão de empréstimo

### Problema de negócio
A empresa precisa analisar se o risco de inadimplência do cliente é baixo o suficiente para realizar o empréstimo de dinheiro. Nesse contexto, será necessário criar um sistema que realiza a classificação de cliente para aprovação de empréstimo.

### Análise exploratória de dados

**Arquitetura de dados** (AD)
- id_cliente: numero de identificação única do cliente
- inad: índice de inadiplência do cliente
- saldo_contas: índice referente aos saldos das contas correntes, poupanças e aplicações do cliente
- class_renda: indice referente a classificação do cliente de acordo com sua renda
- anotacoes: índice referente a anotações externas às empresas, tais como SERASA/SPC/BACEN
- risco: risco do cliente atribuído dado as variáveis de análise do cliente

In [13]:
# bibliotecas
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from joblib import dump
import time


In [3]:
# carga de dados
dados = pd.read_csv("dados/risco.csv")

dados.head()

,id_cliente,anot_cadastrais,indice_inad,class_renda,saldo_contas,Risco
0,0,5.1,3.5,1.4,0.2,Risco_Alto
1,1,4.9,3.0,1.4,0.2,Risco_Alto
2,2,4.7,3.2,1.3,0.2,Risco_Alto
3,3,4.6,3.1,1.5,0.2,Risco_Alto
4,4,5.0,3.6,1.4,0.2,Risco_Alto


In [ ]:
# verifica se há dados duplicados
dados.duplicated().sum()

In [ ]:
# verifica se há dados vazios
dados.isna().sum()

In [ ]:
# verifica os tipos de dados
dados.dtypes

In [ ]:
# verifica os dados de cada coluna
dados.Risco.value_counts()

In [ ]:
# transforma os dados da variável "risco" (label encoder) 
dados['Risco'] = dados['Risco'].map({"Risco_Baixo" : 0, "Risco_Medio" : 1, "Risco_Alto" : 2})

In [ ]:
# correlação dos dados
#display(dados.corr())

sns.heatmap(dados.corr(), annot=True)



In [ ]:
# 
dados.describe()

sns.histplot(dados, x='Risco', y='saldo_contas')
sns.boxplot(dados, x='Risco', y='saldo_contas')


In [ ]:
sns.pairplot(dados, kind="reg", hue='Risco')

### Pré-processamento de dados

In [4]:
#remove colunas desnecessárias
dados.drop(columns=['id_cliente'], inplace=True)
dados

,anot_cadastrais,indice_inad,class_renda,saldo_contas,Risco
0,5.1,3.5,1.4,0.2,Risco_Alto
1,4.9,3.0,1.4,0.2,Risco_Alto
2,4.7,3.2,1.3,0.2,Risco_Alto
3,4.6,3.1,1.5,0.2,Risco_Alto
4,5.0,3.6,1.4,0.2,Risco_Alto
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Risco_Baixo
146,6.3,2.5,5.0,1.9,Risco_Baixo
147,6.5,3.0,5.2,2.0,Risco_Baixo
148,6.2,3.4,5.4,2.3,Risco_Baixo


In [6]:
#separacao das variáveis independentes e a variável dependente (target)

x = dados.drop(columns='Risco')
y = dados['Risco']


In [7]:
# Separa os dados de treino e de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7)

x_test.shape

(45, 4)

In [15]:
# padroniza os dados de treino (fit) e de teste (transform) com StandardScaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_test.shape

dump(scaler, "standard_scaler.joblib")



['standard_scaler.joblib']

### Criação do modelo preditivo

In [11]:
#função que printa os resultados
def imprime_resultado(nome, acuracia, confusion_matrix):
    print("\n\n=========================")
    print(f"Modelo: {nome}")
    print(f"Acuracia: {acuracia}")
    print(confusion_matrix)
    

In [ ]:
from sklearn.svm import SVC

#cria uma avaliação de qual melhor modelo usar
dic_model = {
             #"SGDClassifier" : SGDClassifier(),
             #"LogisticRegression": LogisticRegression(),
             #"RandonForestClassifier" : RandomForestClassifier(), 
             "KNeighborsClassifier" : KNeighborsClassifier(),
             #"GaussianNB" : GaussianNB(),
             #"XGBClassifier" : XGBClassifier(),
             "SVC" : SVC()}


for chave, model in dic_model.items():
    tempo_inicial = time.time()
    model.fit(x_train, y_train)
    
    y_predict = model.predict(x_test)
    
    acuracia = accuracy_score(y_test, y_predict)
    cm = confusion_matrix(y_test, y_predict)
    tempo_final = time.time()
    imprime_resultado(chave, acuracia, cm)
    print(f"Tempo gasto: {tempo_final - tempo_inicial :.2f} segundos")
    

In [9]:
# modelo escolhido pela melhor perfomance (acuracia e tempo)
model = KNeighborsClassifier(
    n_neighbors = 5 #default 5
    ,weights = 'distance' #'uniform’, ‘distance'
    ,leaf_size=30 #default 30
)


tempo_inicial = time.time()
model.fit(x_train, y_train)

y_predict = model.predict(x_test)



### Avaliação do modelo preditivo

In [12]:
acuracia = accuracy_score(y_test, y_predict)
cm = confusion_matrix(y_test, y_predict)
tempo_final = time.time()
imprime_resultado('KNeighborsClassifier', acuracia, cm)
print(f"Tempo gasto: {tempo_final - tempo_inicial :.2f} segundos")





Modelo: KNeighborsClassifier
Acuracia: 0.9555555555555556
[[12  0  0]
 [ 0 15  2]
 [ 0  0 16]]
Tempo gasto: 37.60 segundos


In [14]:
# exporta o modelo
dump(model, "model_class_risco.joblib")


['model_class_risco.joblib']